In [ ]:
from ultralytics import YOLO
import cv2
import easyocr
import csv
import os

''' Comando para entrenar el modelo 

yolo detect train data=dataset.yaml model=yolo11s.pt epochs=150 imgsz=640 batch=16 
device=0 degrees=10 translate=0.1 scale=0.2 
shear=0.05 perspective=0.0 flipud=0.0 fliplr=0.5 mosaic=True mixup=True


'''

main_model = YOLO("./modelo/yolo11s.pt")
main_model.to("cuda")

plate_model = YOLO("./runs/detect/train11/weights/best.pt")
plate_model.to("cuda")

reader = easyocr.Reader(['en'], gpu=True)

video_path = "./videos_prueba/video_por_delante.mp4"
cap = cv2.VideoCapture(video_path)

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(
    "videos_procesados/ocr_delante_resultado_train11.avi",
    cv2.VideoWriter_fourcc(*"XVID"),
    fps,
    (frame_width, frame_height)
)

csv_path = "videos_procesados/detecciones_video_delante.csv"
os.makedirs(os.path.dirname(csv_path) or ".", exist_ok=True)
with open(csv_path, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([
        "frame", "tipo_objeto", "confianza", "tracking_id",
        "x1", "y1", "x2", "y2",
        "matricula_conf", "mx1", "my1", "mx2", "my2", "texto_matricula"
    ])


frame_count = 0
tracker_id = 0
person_ids = set()
vehicle_ids = set()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    current_second = frame_count / fps
    print(f"Procesando segundo: {current_second:.2f}s", end="\r")

    results = main_model.track(frame, classes=[0, 1, 2, 3, 5, 7], persist=True, conf=0.5, device=0)

    if results[0].boxes.id is not None:
        ids = results[0].boxes.id.int().tolist()
    else:
        ids = [None] * len(results[0].boxes.xyxy)

    for box, cls, conf, obj_id in zip(
        results[0].boxes.xyxy,
        results[0].boxes.cls,
        results[0].boxes.conf,
        ids
    ):
        x1, y1, x2, y2 = map(int, box[:4])
        confidence = float(conf)
        class_id = int(cls)
        tracking_id = obj_id if obj_id is not None else tracker_id
        tracker_id += 1

        class_name = main_model.model.names[class_id]

        if class_name == "person":
            color = (0, 0, 255)
            person_ids.add(tracking_id)
        else:
            color = (255, 0, 0)
            vehicle_ids.add(tracking_id)

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"{class_name} {confidence:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)


        plate_text, plate_conf = "", 0.0
        px1 = py1 = px2 = py2 = 0
        if class_name != "person":
            vehicle_crop = frame[y1:y2, x1:x2]
            plate_results = plate_model(vehicle_crop, device=0, conf=0.5)

            for plate_box, plate_conf_t in zip(
                plate_results[0].boxes.xyxy, plate_results[0].boxes.conf
            ):
                px1, py1, px2, py2 = map(int, plate_box[:4])
                px1 += x1
                py1 += y1
                px2 += x1
                py2 += y1
                plate_conf = float(plate_conf_t)


                plate_crop = frame[py1:py2, px1:px2]
                ocr_result = reader.readtext(plate_crop)
                plate_text = " ".join([res[1] for res in ocr_result])

                cv2.rectangle(frame, (px1, py1), (px2, py2), (0, 255, 0), 2)
                cv2.putText(frame, f"{plate_text} ({plate_conf:.2f})",
                            (px1, py1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                            0.6, (0, 255, 0), 2)

        with open(csv_path, mode='a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                frame_count, class_name, confidence, tracking_id,
                x1, y1, x2, y2,
                plate_conf, px1, py1, px2, py2, plate_text
            ])

    cv2.putText(frame, f"Personas: {len(person_ids)}  Vehiculos: {len(vehicle_ids)}",
                (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow("Matriculas", frame)
    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("\n Procesamiento finalizado")
print(f"CSV guardado en: {csv_path}")


In [ ]:
from ultralytics import YOLO
import cv2


face_model = YOLO("modelos/yolov8n_100e.pt")
face_model.to("cuda")


video_path = "./videos_procesados/ocr_detras_resultado_train11.avi"
output_path = "./videos_procesados/ocr_detras_resultado_train11_caras_pixeladas.avi"

cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(
    output_path,
    cv2.VideoWriter_fourcc(*"XVID"),
    fps,
    (frame_width, frame_height)
)


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = face_model(frame, conf=0.5, device=0)

    for box in results[0].boxes.xyxy:
        x1, y1, x2, y2 = map(int, box[:4])


        face = frame[y1:y2, x1:x2]

        if face.size > 0:
            face = cv2.resize(face, (4, 4), interpolation=cv2.INTER_LINEAR)
            face = cv2.resize(face, (x2 - x1, y2 - y1), interpolation=cv2.INTER_NEAREST)
            frame[y1:y2, x1:x2] = face

    out.write(frame)
    cv2.imshow("Pixelado de caras", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video procesado y guardado en {output_path}")


In [ ]:
from ultralytics import YOLO
import cv2
import easyocr
import pytesseract
import os
import csv
import json
import time
from difflib import SequenceMatcher
import matplotlib.pyplot as plt
import pandas as pd


image_folder = "./fotos_para_ocrs"
ground_truth_json = "./etiquetas_fotos_ocrs.json"
csv_output = "comparativa_ocr_metricas_tiempo.csv"



plate_model = YOLO("./runs/detect/train11/weights/best.pt")
plate_model.to("cuda")

reader = easyocr.Reader(['en'], gpu=True)

def normalize_text(text):
    return text.replace(" ", "").upper()

def char_accuracy(true_text, pred_text):
    if not true_text or not pred_text:
        return 0.0
    return SequenceMatcher(None, true_text, pred_text).ratio()


with open(ground_truth_json, "r", encoding="utf-8") as f:
    ground_truth = json.load(f)

with open(csv_output, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow([
        "imagen", "gt_text",
        "easyocr_text", "tesseract_text",
        "easy_exact", "tess_exact",
        "easy_acc", "tess_acc",
        "easy_time(s)", "tess_time(s)"
    ])

for filename in os.listdir(image_folder):
    if not filename.lower().endswith((".jpg", ".png", ".jpeg")):
        continue

    gt_text = ground_truth.get(filename, "").strip()
    image_path = os.path.join(image_folder, filename)
    image = cv2.imread(image_path)

    results = plate_model(image, conf=0.5, device=0)
    if len(results[0].boxes.xyxy) == 0:
        print(f"[!] No se detectó matrícula en {filename}")
        continue

    x1, y1, x2, y2 = map(int, results[0].boxes.xyxy[0])
    crop = image[y1:y2, x1:x2]

    # OCR EasyOCR
    start_easy = time.time()
    easy_result = reader.readtext(crop)
    easy_time = time.time() - start_easy
    easy_text = " ".join([res[1] for res in easy_result]).strip()

    # OCR Tesseract
    start_tess = time.time()
    tesseract_text = pytesseract.image_to_string(crop, config="--psm 7").strip()
    tess_time = time.time() - start_tess

    gt_text_n = normalize_text(gt_text)
    easy_text_n = normalize_text(easy_text)
    tess_text_n = normalize_text(tesseract_text)

    easy_exact = 1 if easy_text_n == gt_text_n else 0
    tess_exact = 1 if tess_text_n == gt_text_n else 0

    easy_acc = char_accuracy(gt_text_n, easy_text_n)
    tess_acc = char_accuracy(gt_text_n, tess_text_n)

    print(f"{filename} → GT: {gt_text_n} | Easy: {easy_text_n} ({easy_acc:.2f}) | Tess: {tess_text_n} ({tess_acc:.2f})")

    with open(csv_output, mode='a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([
            filename, gt_text_n,
            easy_text_n, tess_text_n,
            easy_exact, tess_exact,
            round(easy_acc, 3), round(tess_acc, 3),
            round(easy_time, 3), round(tess_time, 3)
        ])

print("\n Comparativa finalizada.")
print(f"Resultados guardados en: {csv_output}")

df = pd.read_csv(csv_output)

easy_exact_mean = df["easy_exact"].mean() * 100
tess_exact_mean = df["tess_exact"].mean() * 100

easy_acc_mean = df["easy_acc"].mean() * 100
tess_acc_mean = df["tess_acc"].mean() * 100

easy_time_mean = df["easy_time(s)"].mean()
tess_time_mean = df["tess_time(s)"].mean()

# Gráfico 1: Exactitud total
plt.figure(figsize=(6, 4))
plt.bar(["EasyOCR", "Tesseract"], [easy_exact_mean, tess_exact_mean])
plt.ylabel("Porcentaje de aciertos (%)")
plt.title("Aciertos exactos por OCR")
plt.ylim(0, 100)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.savefig("grafico_aciertos.png")
plt.show()

# Gráfico 2: Precisión por caracteres
plt.figure(figsize=(6, 4))
plt.bar(["EasyOCR", "Tesseract"], [easy_acc_mean, tess_acc_mean], color=["orange", "green"])
plt.ylabel("Precisión de caracteres (%)")
plt.title("Accuracy media de caracteres")
plt.ylim(0, 100)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.savefig("grafico_accuracy.png")
plt.show()

# Gráfico 3: Tiempo medio
plt.figure(figsize=(6, 4))
plt.bar(["EasyOCR", "Tesseract"], [easy_time_mean, tess_time_mean], color=["purple", "blue"])
plt.ylabel("Tiempo medio (s)")
plt.title("Velocidad promedio de OCR")
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.savefig("grafico_tiempo.png")
plt.show()

print("\n Gráficos guardados")
